In [10]:
# 1. 구글 코랩 메뉴에서 [런타임] → [런타임 다시 시작] 클릭!

# 2. 아래 코드만 복사해서 실행
# 1. 구글 코랩 메뉴에서 [런타임] → [런타임 다시 시작] 클릭!

# 2. 아래 코드만 복사해서 실행
!pip install pykrx pandas numpy scikit-learn matplotlib seaborn -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pykrx import stock
import time
import os
import pickle
import warnings
warnings.filterwarnings('ignore')
from concurrent.futures import ThreadPoolExecutor, as_completed # 추가

print(f"✅ numpy: {np.__version__}")
print(f"✅ pandas: {pd.__version__}")
print("✅ 준비 완료!")


# ============================================================
# 1. 캐시 시스템 (가장 중요!)
# ============================================================
class StockDataCache:
    def __init__(self, cache_dir='stock_cache'):
        self.cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)

    def get_cache_path(self, ticker, date):
        return f"{self.cache_dir}/{ticker}_{date}.pkl"

    def save(self, ticker, date, data):
        with open(self.get_cache_path(ticker, date), 'wb') as f:
            pickle.dump(data, f)

    def load(self, ticker, date):
        path = self.get_cache_path(ticker, date)
        if os.path.exists(path):
            with open(path, 'rb') as f:
                return pickle.load(f)
        return None

cache = StockDataCache()

# ============================================================
# 2. 단일 종목 데이터 수집 (캐시 적용)
# ============================================================
def fetch_ticker_data(ticker, date):
    """캐시 확인 후 데이터 수집"""

    # 캐시 확인
    cached = cache.load(ticker, date)
    if cached:
        return cached

    try:
        name = stock.get_market_ticker_name(ticker)

        # 1. PER, PBR
        df_fund = stock.get_market_fundamental(date, date, ticker)
        if df_fund.empty:
            return None

        fund_row = df_fund.iloc[0]
        per = fund_row['PER'] if 'PER' in fund_row.index else np.nan
        pbr = fund_row['PBR'] if 'PBR' in fund_row.index else np.nan

        if pd.isna(per) or pd.isna(pbr) or per <= 0:
            return None

        # 2. 가격 데이터 (필수!)
        start_date = (datetime.strptime(date, '%Y%m%d') - timedelta(days=200)).strftime('%Y%m%d')
        df_price = stock.get_market_ohlcv_by_date(start_date, date, ticker)

        if len(df_price) < 100:
            return None

        # 모멘텀 계산
        momentum_1m = (df_price['종가'].iloc[-1] / df_price['종가'].iloc[-22] - 1) * 100
        momentum_3m = (df_price['종가'].iloc[-1] / df_price['종가'].iloc[-66] - 1) * 100
        momentum_6m = (df_price['종가'].iloc[-1] / df_price['종가'].iloc[-132] - 1) * 100 if len(df_price) > 132 else 0
        volatility = df_price['등락률'].tail(60).std()

        result = {
            '티커': ticker,
            '종목': name,
            'PER': round(per, 2),
            'PBR': round(pbr, 2),
            '모멘텀_1m': round(momentum_1m, 2),
            '모멘텀_3m': round(momentum_3m, 2),
            '모멘텀_6m': round(momentum_6m, 2),
            '변동성': round(volatility, 2),
            '현재가': df_price['종가'].iloc[-1]
        }

        # 캐시 저장
        cache.save(ticker, date, result)
        return result

    except Exception as e:
        return None

# ============================================================
# 3. 병렬 처리로 월별 데이터 수집
# ============================================================
def collect_month_data_parallel(date, max_workers=10):
    """한 달치 데이터 병렬 수집"""

    print(f"📅 {date} 수집중...")

    # 종목 리스트
    tickers = stock.get_market_ticker_list(date, market="KOSPI")[:300]

    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_ticker_data, ticker, date): ticker
                  for ticker in tickers}

        for future in as_completed(futures):
            result = future.result()
            if result:
                result['기준일'] = date
                results.append(result)

    df = pd.DataFrame(results)
    print(f"  ✅ {date}: {len(df)}개 종목")
    return df

# ============================================================
# 4. 2025년 전체 데이터 수집 (병렬 + 캐시)
# ============================================================
def collect_2025_complete():
    """2025년 전체 데이터 수집 (캐시 있으면 1초!)"""

    # 이미 전체 캐시가 있는지 확인
    cache_file = '2025_complete_cache.pkl'
    if os.path.exists(cache_file):
        print("📦 전체 캐시 발견! 즉시 로딩...")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)

    months = [
        '20250131', '20250228', '20250331', '20250430',
        '20250530', '20250630', '20250731', '20250829',
        '20250930', '20251031', '20251128', '20251230'
    ]

    all_data = []
    for month in months:
        df_month = collect_month_data_parallel(month, max_workers=10)
        if len(df_month) > 0:
            all_data.append(df_month)

    final_df = pd.concat(all_data, ignore_index=True)

    # 전체 캐시 저장
    with open(cache_file, 'wb') as f:
        pickle.dump(final_df, f)

    return final_df

# ============================================================
# 5. 다음달 수익률 추가 (진짜 target!)
# ============================================================
def add_future_returns(df):
    """각 종목의 다음달 수익률 계산"""

    results = []
    grouped = df.groupby('티커')

    for ticker, group in grouped:
        group = group.sort_values('기준일')

        for i in range(len(group)-1):
            current = group.iloc[i]
            next_row = group.iloc[i+1]

            # 다음달 수익률
            future_return = (next_row['현재가'] / current['현재가'] - 1) * 100

            row_dict = current.to_dict()
            row_dict['다음달수익률'] = round(future_return, 2)
            row_dict['target'] = 1 if future_return > 0 else 0

            results.append(row_dict)

    return pd.DataFrame(results)

# ============================================================
# 실행!
# ============================================================
print("="*60)
print("🚀 초고속 데이터 수집 시스템 (가격 데이터 포함)")
print("="*60)

# 1. 2025년 데이터 수집 (처음만 3-5분, 다음부터 1초)
start = time.time()
df_2025 = collect_2025_complete()
print(f"\n✅ 2025년 데이터 수집 완료! {len(df_2025)}개")
print(f"⏱️ 소요시간: {time.time()-start:.1f}초")

# 실행 (10-15분 → 3-5분으로 단축!)
# df_2025 = collect_2025_data_fast()

# 실행 (처음 한 번만)
# df_2025 = collect_2025_monthly_data()
# ============================================================
# 2단계: 2025년 데이터로 투자 전략 수립
# ============================================================
# 2단계: AI 모델 학습 실행
print("="*60)
print("🤖 AI 모델 학습 시작")
print("="*60)

# 먼저 다음달 수익률 추가
print("\n📊 다음달 수익률 계산중...")
df_train = add_future_returns(df_2025)
print(f"✅ 학습 데이터 준비 완료! {len(df_train)}개 샘플")

# AI 모델 학습
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# 특성과 타겟 분리
feature_cols = ['PER', 'PBR', '모멘텀_1m', '모멘텀_3m', '변동성']
X = df_train[feature_cols].values
y = df_train['target'].values

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 랜덤포레스트 모델 학습
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42
)

model.fit(X_train, y_train)

# 검증
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)

print(f"\n🤖 AI 모델 학습 완료!")
print(f"✅ 검증 정확도: {accuracy:.2%}")

# 특성 중요도
importance = pd.DataFrame({
    '특성': feature_cols,
    '중요도': model.feature_importances_
}).sort_values('중요도', ascending=False)

print("\n📊 특성 중요도:")
print(importance.to_string(index=False))
# 3단계: 2026년 1월 투자 종목 선정
print("\n" + "="*60)
print("💰 2026년 1월 투자 종목 선정")
print("="*60)

def select_stocks_jan2026(model, scaler, features):
    """
    2026년 1월 31일 데이터로 투자 종목 선정
    """
    date = '20260130'  # 1월 마지막 거래일
    tickers = stock.get_market_ticker_list(date, market="KOSPI")[:300]

    stock_data = []

    for ticker in tickers:
        try:
            name = stock.get_market_ticker_name(ticker)
            df_fund = stock.get_market_fundamental(date, date, ticker)

            if df_fund.empty:
                continue

            fund_row = df_fund.iloc[0]
            per = fund_row['PER'] if 'PER' in fund_row.index else np.nan
            pbr = fund_row['PBR'] if 'PBR' in fund_row.index else np.nan

            if pd.isna(per) or pd.isna(pbr) or per <= 0:
                continue

            # 가격 데이터
            start_date = (datetime.strptime(date, '%Y%m%d') - timedelta(days=200)).strftime('%Y%m%d')
            df_price = stock.get_market_ohlcv_by_date(start_date, date, ticker)

            if len(df_price) < 100:
                continue

            momentum_1m = (df_price['종가'].iloc[-1] / df_price['종가'].iloc[-22] - 1) * 100
            momentum_3m = (df_price['종가'].iloc[-1] / df_price['종가'].iloc[-66] - 1) * 100
            volatility = df_price['등락률'].tail(60).std()

            stock_data.append({
                '종목': name,
                '티커': ticker,
                'PER': per,
                'PBR': pbr,
                '모멘텀_1m': momentum_1m,
                '모멘텀_3m': momentum_3m,
                '변동성': volatility,
                '1월31일_가격': df_price['종가'].iloc[-1]
            })

        except:
            continue

    df_jan = pd.DataFrame(stock_data)
    print(f"✅ {len(df_jan)}개 종목 데이터 수집 완료")

    # AI 점수 계산
    X_jan = scaler.transform(df_jan[features].values)
    df_jan['AI_점수'] = model.predict_proba(X_jan)[:, 1]

    # 종합 점수
    df_jan['종합점수'] = df_jan['AI_점수']

    # 상위 20개 종목 선정
    top_20 = df_jan.nlargest(20, '종합점수')

    print("\n" + "="*70)
    print("💰 2026년 1월 31일 투자 선정 종목 TOP 20")
    print("="*70)
    print(top_20[['종목', 'PER', 'PBR', '모멘텀_1m', 'AI_점수']].head(20).to_string(index=False))

    return top_20

# 실행
portfolio_jan = select_stocks_jan2026(model, scaler, feature_cols)
# 4단계: 2026년 2월 13일 성과 확인
print("\n" + "="*60)
print("📊 2026년 2월 13일 성과 확인")
print("="*60)

def check_feb_results(portfolio_jan):
    """1월에 선정한 종목들의 2월 13일 성과 확인"""

    results = []
    total_return = 0

    for idx, row in portfolio_jan.iterrows():
        ticker = row['티커']
        name = row['종목']
        buy_price = row['1월31일_가격']

        try:
            # 2월 13일 가격
            df_feb = stock.get_market_ohlcv_by_date('20260213', '20260213', ticker)

            if df_feb.empty:
                continue

            feb_price = df_feb['종가'].iloc[0]
            return_pct = (feb_price / buy_price - 1) * 100

            results.append({
                '종목': name,
                '매수가(1/31)': buy_price,
                '현재가(2/13)': feb_price,
                '수익률(%)': round(return_pct, 2),
                'PER': row['PER'],
                'PBR': row['PBR']
            })

        except:
            continue

    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values('수익률(%)', ascending=False)

    print("\n📈 개별 종목 성과:")
    print(df_results.head(10).to_string(index=False))

    print(f"\n{'='*70}")
    print(f"📊 최종 결과")
    print(f"{'='*70}")
    print(f"✅ 분석 가능 종목: {len(df_results)}개")
    print(f"✅ 평균 수익률: {df_results['수익률(%)'].mean():.2f}%")
    print(f"✅ 상승 종목: {len(df_results[df_results['수익률(%)'] > 0])}개")
    print(f"✅ 하락 종목: {len(df_results[df_results['수익률(%)'] < 0])}개")

    return df_results

# 실행
feb_results = check_feb_results(portfolio_jan)

✅ numpy: 1.26.4
✅ pandas: 2.2.2
✅ 준비 완료!
🚀 초고속 데이터 수집 시스템 (가격 데이터 포함)
📦 전체 캐시 발견! 즉시 로딩...

✅ 2025년 데이터 수집 완료! 2317개
⏱️ 소요시간: 0.0초
🤖 AI 모델 학습 시작

📊 다음달 수익률 계산중...
✅ 학습 데이터 준비 완료! 2096개 샘플

🤖 AI 모델 학습 완료!
✅ 검증 정확도: 57.14%

📊 특성 중요도:
    특성      중요도
모멘텀_3m 0.229152
   변동성 0.207353
모멘텀_1m 0.198560
   PER 0.195049
   PBR 0.169885

💰 2026년 1월 투자 종목 선정
✅ 198개 종목 데이터 수집 완료

💰 2026년 1월 31일 투자 선정 종목 TOP 20
       종목   PER   PBR    모멘텀_1m    AI_점수
       남성  5.70  0.24 -7.744875 0.803133
    LX세미콘  6.82  0.82 11.291963 0.791707
      넥스틸  8.43  0.63 23.380900 0.751905
      광전자  9.24  0.41  7.158713 0.731011
       경농  9.38  0.64 -3.007519 0.730428
     KG스틸  4.20  0.28  5.046729 0.722760
     고려산업 14.85  0.55 -4.724409 0.722509
 SNT다이내믹스 11.54  1.40  6.330472 0.709127
       금비 11.13  0.49 -5.609284 0.709124
     대원제약 17.32  0.90 -3.179916 0.709033
    농심홀딩스  5.49  0.34 -2.209945 0.705475
    대유에이텍  9.11  1.07 -6.820276 0.698402
 KISCO홀딩스  9.46  0.26 -4.174950 0.696681
     KTcs 19.68  0.54 -4.